In [1]:
#import dependencies
#from config import api_key
import pandas as pd
#from sodapy import Socrata
from ipywidgets import interact, interactive
import ipywidgets as widgets
#from ipyleaflet import Map
import requests 
#import matplotlib.pyplot as plt
#import matplotlib.cm
import folium

#from mpl_toolkits.basemap import Basemap
#from mpl_toolkits.basemap import Basemap
#from matplotlib.patches import Polygon
#from matplotlib.collections import PatchCollection
#from matplotlib.colors import Normalize

#client = Socrata('data.transportation.gov', api_key)

#results = client.get('38pg-dj5d', city1='Austin, TX')

In [2]:
def get_data(year=2007, airport_1='AUS', data_request_type=0):
    #load the data
    df =  pd.read_csv('./resources/data/Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv')

    #filter where the origin airport is Austin, TX for 2007 and above
    df1 = df[(df['Year'] == year) &(df['airport_1'] == airport_1)]

    #pull out the lat and log for the city pairs
    pd.options.mode.chained_assignment = None
    df1['city1_lat'] = pd.to_numeric(df1['Geocoded_City1'].str.split('\n').str[1].str.split(',').str[0].str.replace('(', ''))
    df1['city1_lng'] = pd.to_numeric(df1['Geocoded_City1'].str.split('\n').str[1].str.split(',').str[1].str.replace(')', ''))
    df1['city2_lat'] = pd.to_numeric(df1['Geocoded_City2'].str.split('\n').str[1].str.split(',').str[0].str.replace('(', ''))
    df1['city2_lng'] = pd.to_numeric(df1['Geocoded_City2'].str.split('\n').str[1].str.split(',').str[1].str.replace(')', ''))
    
    
    df2 = df1[['Year', 'quarter', 'city1', 'city2', 'airport_1', 'airport_2', 'nsmiles'
         , 'passengers', 'fare', 'carrier_lg', 'city1_lat', 'city2_lat', 'city2_lat', 'city2_lng' ]]
    
    if data_request_type == 0:
        #All records
        return df2
    if data_request_type == 1:
        #Average Fare and Total Passengers by Year
        return df2.groupby(['Year', 'city1', 'city2']).agg({'fare': 'mean', 'passengers': 'sum'}).reset_index()            
    if data_request_type == 2:
        #Unique list of carriers for the Year
        return pd.DataFrame(df2['carrier_lg'].sort_values().unique()).rename(columns={0: 'carrier_lg'})            
    if data_request_type == 3:
        #List of segments for the year
        return pd.DataFrame(df2['city2'].sort_values().unique()).rename(columns={0: 'city2'})

In [3]:
"""
Function that gets a unique list of cities 

@param1: column: the name of the column

returns: a list of unique cities
"""
def get_cities(column):
    #return pd.read_csv('./resources/data/Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv')[column].sort_values().unique()
    return pd.read_csv(f'./resources/data/{column}.csv')[column]

In [4]:
#save the data to a csv since the original dataset it too large to save on github
#pd.DataFrame(get_cities('city2')).rename(columns={0: 'city2'}).to_csv('./resources/data/city2.csv', index=False)

In [5]:
"""
Function that builds a dataframe based on input param values

@param1: year
@param2: city1: Typically Austin, TX
@param3: data_request_type
         0: for all records
         1: Average fare and total passengers by year
         2: List of carriers for the year
         3: List of segments for the year
"""
def build_df(year, city1, data_request_type):#, city2):
    try:
        base_url = f'https://data.transportation.gov/resource/38pg-dj5d.json?$$app_token={api_key}'
        params = {'city1': f'{city1}'
                  , 'year': f'{year}'
                  #, 'city2': f'{city2}'
                 }

        results = requests.get(base_url, params).json()
        df = pd.DataFrame(results)
        df['fare'] = pd.to_numeric(df['fare'])
        df['passengers'] = pd.to_numeric(df['passengers'])
        df['quarter'] = pd.to_numeric(df['quarter'])
        df['year'] = pd.to_numeric(df['year'])
        
        df1 = df[['carrier_lg'
                   #, 'carrier_low'
                   , 'city1'
                   , 'city2'
                   , 'location_2'
                   , 'fare'
                   , 'passengers'
                   , 'quarter'
                   , 'year']].sort_values(['city2', 'year', 'quarter'])
        if data_request_type == 0:
            #All records
            return df1
        if data_request_type == 1:
            #Average Fare and Total Passengers by Year
            return df1.groupby(['year', 'city1', 'city2']).agg({'fare': 'mean', 'passengers': 'sum'}).reset_index()            
        if data_request_type == 2:
            #Unique list of carriers for the Year
            return pd.DataFrame(df['carrier_lg'].sort_values().unique()).rename(columns={0: 'carrier_lg'})            
        if data_request_type == 3:
            #List of segments for the year
            return pd.DataFrame(df['city2'].sort_values().unique()).rename(columns={0: 'city2'})
    except KeyError:
        print("Looks like there is no data for this city pair, try another")

In [6]:
interact(build_df, year=(2007, 2017), city1=get_cities('city1')
         , data_request_type={'All records': 0
                              , 'Average fare and total passengers by year': 1
                              , 'List of carriers for the year': 2
                              , 'List of segments for the year': 3
                             })


interactive(children=(IntSlider(value=2012, description='year', max=2017, min=2007), Dropdown(description='cit…

<function __main__.build_df(year, city1, data_request_type)>

In [7]:
interact(get_data, year=(1997, 2017), airport_1='AUS'
         , data_request_type={'All records': 0
                              , 'Average fare and total passengers by year': 1
                              , 'List of carriers for the year': 2
                              , 'List of segments for the year': 3
                             })

interactive(children=(IntSlider(value=2007, description='year', max=2017, min=1997), Text(value='AUS', descrip…

<function __main__.get_data(year=2007, airport_1='AUS', data_request_type=0)>

In [8]:
folium_map = folium.Map(location=[30.266926, -97.750519],
                        zoom_start=5,
                        tiles="CartoDB dark_matter")

folium.CircleMarker(location=[30.266926, -97.750519],fill=True).add_to(folium_map)
folium_map

In [9]:
def get_passenger_count_by_destination():
     folium_map = folium.Map(location=[30.266926, -97.750519],
                            zoom_start=5,
                            tiles="CartoDB dark_matter",
                            width='50%')

    # for each row in the data, add a cicle marker
    for index, row in trip_counts.iterrows():
        # calculate net departures
        net_departures = (row["Departure Count"]-row["Arrival Count"])
        
        # generate the popup message that is shown on click.
        popup_text = "{}<br> total departures: {}<br> total arrivals: {}<br> net departures: {}"
        popup_text = popup_text.format(row["Start Station Name"],
                          row["Arrival Count"],
                          row["Departure Count"],
                          net_departures)
        
        # radius of circles
        radius = net_departures/20
        
        # choose the color of the marker
        if net_departures>0:
            # color="#FFCE00" # orange
            # color="#007849" # green
            color="#E37222" # tangerine
        else:
            # color="#0375B4" # blue
            # color="#FFCE00" # yellow            
            color="#0A8A9F" # teal
        
        # add marker to the map
        folium.CircleMarker(location=(row["Start Station Latitude"],
                                      row["Start Station Longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=True).add_to(folium_map)
    return folium_map

IndentationError: unindent does not match any outer indentation level (<ipython-input-9-bf0053dce38b>, line 8)